# NLP

Fichier pris en entrée :
- clean_{date}.json -> provient du notebook **nettoyeur**

Fichiers de sortie produits par le notebook :
- NLP_{date}_{type_pipeline}_{nombre_resultats}.html -> **fichier HTML des résultats de la NER**

- NLP_{date}_{type_pipeline}_{nombre_resultats}.csv -> **tableau avec autant de lignes que de résultats et deux colonnes : titre et pers**
**-> pers** = entitées nommées "personnage" issues de spacy pour lesquelles un traitement de nettoyage a été effectué :
    - suppression des termes inférieur à 2 caractères
    - suppression des doublons malgré d'éventuelles coquilles d'OCR dans la graphie du nom -> la graphie retenue est celle qui revient le plus de fois au sein du CR

In [2]:
import pandas as pd
import spacy
from spacy import displacy
from collections import Counter
# Pipelines au choix small, medium, large (du - au + précis)
# import fr_core_news_sm
# import fr_core_news_md
import fr_core_news_lg
nlp = fr_core_news_lg.load()
from datetime import date
import time
import json
import itertools
import regex as re
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
# Variables à changer par l'utilisateur
source_json = 'results/clean_w_names_24-01-23.json'
pipeline = "fr_core_news_lg" # /!\ PENSER À CHANGER PLUS HAUT

# Importation des données depuis le JSON
df = pd.read_json(source_json, orient='index')

# Nombre de résultats dans la réponse produite, varie en fonction des essais
nb_a_traiter = len(df.index)
# nb_a_traiter = 100

# Variable permettant de compter puis d'inscrire le nombre de "personnes" détéctées par la NER dans le nom du fichier de sortie final (pour contrôle)
count_names = 0

In [4]:
# Définition de l'en tête général
date = time.strftime("%d-%m-%y")

# HTML description des labels
labels_description = """
<p><strong>PERSON:</strong> People, including fictional.&emsp;<strong>NORP:</strong> Nationalities or religious or political groups.&emsp;
<strong>FAC:</strong> Buildings, airports, highways, bridges, etc.&emsp;<strong>ORG:</strong> Companies, agencies, institutions, etc.&emsp;
<strong>GPE:</strong> Countries, cities, states.&emsp;<strong>LOC:</strong> Non-GPE locations, mountain ranges, bodies of water.&emsp;
<strong>PRODUCT:</strong> Objects, vehicles, foods, etc. (Not services.)&emsp;<strong>EVENT:</strong> Named hurricanes, battles, wars, sports events, etc.&emsp;
<strong>WORK_OF_ART:</strong> Titles of books, songs, etc.&emsp;<strong>LAW:</strong> Named documents made into laws.&emsp;<strong>LANGUAGE:</strong> Any named language.&emsp;
<strong>DATE:</strong> Absolute or relative dates or periods.&emsp;<strong>TIME:</strong> Times smaller than a day.&emsp;<strong>PERCENT:</strong> Percentage, including "%".&emsp;
<strong>MONEY:</strong> Monetary values, including unit.&emsp;<strong>QUANTITY:</strong> Measurements, as of weight or distance.&emsp;<strong>ORDINAL:</strong> "first", "second", etc.&emsp;
<strong>CARDINAL:</strong> Numerals that do not fall under another type.</p>
"""

heading = f"""
    <p><strong>date:</strong> {date}</p>
    <p><strong>source:</strong> {source_json}</p>
    <p><strong>pipeline:</strong> {pipeline}</p>
    <p><strong>quantity processessed:</strong> {nb_a_traiter}</p>
    <hr>
    {labels_description}
    """

# (Re)création d'un fichier de sortie propre + en tête
nom_fichier = f"results/NLP_{date}_{pipeline[-2:]}_{nb_a_traiter}r_{count_names}n"

# with open(f"{nom_fichier}.html", "w") as fichier:
#     fichier.write(heading)

In [5]:
# Personnalisation des couleurs du rendu
colors = {
    "PER": "#97C7E8",
    "ORG": "#A4DBA4",
    "GPE": "#F2937C",
    "LOC": "#AE9DF2",
    "EVENT": "#E8BC76",
    "WORK_OF_ART": "#DB99DB",
    "MISC": "#F2A99D",
    "DATE": "#A7F2BD",
    "ORDINAL": "#E8D3A2",
    "CARDINAL": "#E8D3A2",
}

In [6]:
lists = []

# Boucle de constitution : export PER + HTML
# for i in range(nb_a_traiter): # range(len(df.index))
for i in range(nb_a_traiter):
    # NLP par spacy --------------------------------------------------------------------------
    title = df.loc[i, 'area_title']
    text = df.loc[i, 'area_text']
    extracted_names = df.loc[i, 'extracted_names']
    doc = nlp(text)

    # Constitution de la grande liste destinée à être convertie en df ------------------------
    pers = [(ent.text) for ent in doc.ents if ent.label_ == "PER"]
    list = [title, pers, extracted_names]
    lists.append(list)

    # Constitution de l'HTML de contrôle du NLP ----------------------------------------------
    sentence_tokens = len([[token.text for token in sent] for sent in doc.sents])

    # # Génération du rendu displacy
    # html = displacy.render(doc, style="ent", jupyter=False, page=True, options={"colors": colors})

    # # Définition de l'en tête pour chaque résultat
    # headings = f"""
    # <hr>
    # <p><strong>index:</strong> {i}</p>
    # <p><strong>title:</strong> {title}</p>
    # <p><strong>number of sentences:</strong> {sentence_tokens}</p>
    # """

    # # Inscription de l'en tête + inscription de résultat
    # with open(f"{nom_fichier}.html", 'a') as fichier:
    #     fichier.write(headings)
    #     fichier.write(str(html))

In [7]:
# Conversion de la liste vers df
df_PER = pd.DataFrame(lists, columns=['title', 'pers', 'extracted_names'])

In [14]:
before = df_PER['pers'][0]

In [20]:
# Fonction de nettoyage de la liste des entitées reconnues
def clean_names(list):
    patterns = [
        r'^M\.',
        r'^MM\.',
        r'^MMe.',
        r'^MMe ',
        r'^ +',
    ]
    for i in range(len(list)):
        for pattern in patterns:
            list[i] = re.sub(pattern, '', list[i])
    return(list)

# Comptage des termes au sein de la liste
def make_count(list):
    list_count = []
    for i in range(len(list)):
        mot = list[i]
        count = list.count(list[i])
        list_count.append([mot, count])
    return(list_count)

# Fonction de comparaisons des mots 2 à 2 : lorsque plusieurs noms sont similaires, on n'en garde qu'un
# Fonction qui prend une liste en entrée et ressort une liste en sortie
def keep_best_name(list):
    discri_list = [
        r'^[\P{Lu}]',
        r'[0-9]+',
        r'\.$',
        r'\"|\'|\\|\/|«|»'
    ]

    # Pour chaque mot
    temp_list = []
    for i in range(len(list)):
        ref = list[i]

        # On va comparera avec tous autres mots de la liste
        for j in range(len(list)):
            # Si on confronte le mot avec lui même -> on ne fait rien
            if i == j:
                pass
            else:
                # Sinon comparaison du score entre les deux noms
                ratio = fuzz.token_sort_ratio(ref[0], list[j][0])
                partial_ratio = fuzz.partial_ratio(ref[0], list[j][0])
                if ratio > 85 or partial_ratio > 85:
                    # Si le second mot est présent plus de fois, on le retient lui
                    if ref[1] < list[j][1]:
                        ref = list[j]

        # Discrimination des noms de 2 lettres ou moins, ceux qui contiennent de chiffres, qui ne commencent pas par une maj et ceux ne contenant pas de voyelles
        if (ref[0] not in temp_list) & (len(ref[0]) > 2):
            ready = True
            for pattern in discri_list:
                if re.search(pattern, ref[0]):
                    ready = False
                if not re.search('(?i)[aeiouy]+', ref[0]):
                    ready = False
            if ready:
                temp_list.append(ref[0])
    return(temp_list)

In [21]:
# Pour chaque ligne de notre tableau de départ
for i in range(len(df_PER.index)):
    # On appelle la fonction de nettoyage sur nos entitées "personnes"
    list_PER_clean = clean_names(df_PER['pers'][i])
    # Chaque entitée est compté, un nombre d'occurence lui est attribué = [['Personne1', 2], ['Personne2', 3], ['Personne3', 1], ['Personne4', 1]]
    list_PER_count = make_count(list_PER_clean)
    # De tous les noms qui se ressemblent (= coquilles dans la graphie), on ne garde que l'occurence qui est apparue le plus de fois
    best_names = keep_best_name(list_PER_count)
    # Décompte pour contrôle 
    count_names += len(best_names)
    # Màj de notre liste d'entités "personnes" au sein du dataframe
    df_PER.at[i, 'pers'] = best_names

In [22]:
# (Re)création d'un fichier de sortie propre + en tête
nom_fichier = f"results/NLP_{date}_{pipeline[-2:]}_{nb_a_traiter}r_{count_names}n"

# Conversion du df vers csv
df_PER.to_csv(f"{nom_fichier}.csv")

count_names

19448

In [23]:
after = df_PER['pers'][0]
print(f'AVANT : {before}')
print('----------------------')
print(f'APRÈS : {after}')

AVANT : ['P. Vergili Maronis', 'Épigrammata', 'E. Galletier', 'Gailetier', 'Galletier', 'Suétone', 'Virgile', 'fecit Kax', 'Galletier', 'Domitien', 'Néron', 'Virgile', 'Galletier', 'Birt, Gubernatis', 'V', 'Vili', 'Ite hinc', 'Catulle X', 'Sabinus Me', 'Virgile', 'Varius', 'Octavius', 'IX', 'Virgile', 'Corinthiorum', 'Galletier', 'Virgile', 'Catulle', 'Virgile', 'Racine', 'Virgile', 'Galletier', 'Catulle', 'Galletier', 'Birt', 'Gubernatis', 'René Pichon']
----------------------
APRÈS : ['P. Vergili Maronis', 'Épigrammata', 'Galletier', 'Suétone', 'Virgile', 'Domitien', 'Néron', 'Birt, Gubernatis', 'Vili', 'Ite hinc', 'Catulle', 'Sabinus Me', 'Varius', 'Octavius', 'Corinthiorum', 'Racine', 'Birt', 'Gubernatis', 'René Pichon']
